<a href="https://colab.research.google.com/github/A348305/My_Projects_2024/blob/main/My_mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 1. Data Loading and Preprocessing
# Load your datasets
df1 = pd.read_csv("https://raw.githubusercontent.com/A348305/mydatasets/master/ConfLongDemo_JSI.csv")
#print(df1)

In [ ]:
df2 = pd.read_csv("https://raw.githubusercontent.com/A348305/mydatasets/master/kinetic%20data.csv")
#print(df2)

In [ ]:
df3 = pd.read_csv("https://raw.githubusercontent.com/A348305/mydatasets/master/test.xlsb.csv")
#print(df3)

In [ ]:
combined_dataset = pd.concat([df1, df2, df3], axis=1)
# Handling missing values
combined_dataset.fillna(0, inplace=True)
combined_dataset = combined_dataset[(combined_dataset != 0).all(axis=1)]
print(combined_dataset)

In [ ]:
missing_values =combined_dataset.isna().sum()
missing_percent = (combined_dataset.isna().sum() / len(combined_dataset)) * 100
print(missing_values)
print(missing_percent)

In [ ]:
numeric_cols = combined_dataset.select_dtypes(include=['float64', 'int64']).columns
non_numeric_cols = combined_dataset.select_dtypes(exclude=['float64', 'int64']).columns

# Label encoding for non-numeric data
label_encoders = {}
for col in non_numeric_cols:
    # Convert all values in the column to string type
    combined_dataset[col] = combined_dataset[col].astype(str)
    le = LabelEncoder()
    combined_dataset[col] = le.fit_transform(combined_dataset[col])
    label_encoders[col] = le

 #Standardize numeric data
scaler = StandardScaler()
combined_dataset[numeric_cols] = scaler.fit_transform(combined_dataset[numeric_cols])


# Elbow Method to find the optimal number of clusters
inertia_values = []
possible_k_values = range(1, 15)  # Check for up to 14 clusters

for k in possible_k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(combined_dataset[numeric_cols])  # Use only numeric columns for clustering
    inertia_values.append(kmeans.inertia_)

# Plot the Elbow Method graph
plt.figure(figsize=(10, 6))
plt.plot(possible_k_values, inertia_values, marker='o', linestyle='--')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method to Determine Optimal Cluster Number')
plt.grid(True)
plt.show()

# Compute the Correlation Matrix
correlation_matrix = combined_dataset[numeric_cols].corr()

# Visualize the Correlation Matrix using a Heatmap
plt.figure(figsize=(15, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, square=True, linewidths=0.5)
plt.title('Feature Correlation Heatmap')
plt.show()

# Step 3: Apply a machine learning algorithm
# Here, we'll use K-Means clustering as an example
num_clusters = 4  # You can adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
combined_dataset['cluster'] = kmeans.fit_predict(combined_dataset[numeric_cols])  # Use only numeric columns for clustering

# Step 4: Dimensionality reduction (optional)
# You can use PCA or other dimensionality reduction techniques if needed
pca = PCA(n_components=2)
combined_dataset['pca_1'], combined_dataset['pca_2'] = pca.fit_transform(combined_dataset[numeric_cols]).T

# Step 5: Save or analyze the results
# The 'cluster' column now contains cluster labels for each data point
# You can further analyze the clusters or use them for downstream tasks

# Optional: Inverse transform for non-numeric data (if needed)
for col, le in label_encoders.items():
    combined_dataset[col] = le.inverse_transform(combined_dataset[col])

# Optional: Inverse transform for numeric data (if needed)
combined_dataset[numeric_cols] = scaler.inverse_transform(combined_dataset[numeric_cols])

# Save the results to a new CSV file
combined_dataset.to_csv('clustered_combined_dataset.csv', index=False)

In [ ]:
# Visualizing the clusters after PCA
plt.figure(figsize=(10, 7))
sns.scatterplot(x='pca_1', y='pca_2', data=combined_dataset, hue='cluster', palette='viridis', s=60, edgecolor=None, alpha=0.7)
plt.title('K-Means Clustering with PCA Visualization')
plt.show()

In [ ]:

davies_bouldin = davies_bouldin_score(combined_dataset[numeric_cols], combined_dataset['cluster'])
print(f"Davies-Bouldin Score: {davies_bouldin:.3f}")

